In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
import Plots: Plot
gr();

In [ ]:
function make_ax(xlim, ylim)
    p = plot(aspect_ratio=:equal, xlim=xlim, ylim=ylim, xlabel="X", ylabel="Y")
    return p
end

function draw_traj!(xs, p::Plot{T}) where {T}
    # xs is dictionary
    poses = [xs[s] for s in 0:length(xs)-1]
    p = scatter!([e[1] for e in poses], [e[2] for e in poses], markershape=:circle, markersize=1, color="black")
    p = plot!([e[1] for e in poses], [e[2] for e in poses], linewidth=0.5, color="black")
end

function draw_observations!(
        xs::Dict{Int64, Vector{Float64}},
        zlist::Dict{Int64, Vector{Vector{Float64}}},
        p::Plot{T}
        ) where {T}
    for s in 0:length(xs)-1
        if !haskey(zlist, s)
            continue
        end
        v = zlist[s]
        N = size(v)[1]
        for i in 1:N
            x, y, θ = xs[s][1], xs[s][2], xs[s][3]
            obsv = v[i]
            d, ϕ = obsv[2], obsv[3]
            mx = x + d * cos(θ + ϕ)
            my = y + d * sin(θ + ϕ)
            p = plot!([x, mx], [y, my], color="pink", alpha=0.5)
        end
    end
end

function draw_all(
        xlim, ylim,
        xs::Dict{Int64, Vector{Float64}},
        zlist::Dict{Int64, Vector{Vector{Float64}}},
        p::Plot{T}
        ) where {T}
    p = make_ax(xlim, ylim)
    draw_observations!(xs, zlist, p)
    draw_traj!(xs, p)
    return p
end

# hat_xs contains all the keys in 0:length(xs), but z does not (there are some time steps with no observation)
function read_data(fname="log.txt")
    hat_xs = Dict{Int64, Vector{Float64}}()
    zlist = Dict{Int64, Vector{Vector{Float64}}}()
    delta = 0.0
    us = Dict{Int64, Vector{Float64}}()

    fd = open(fname, "r")
    lines = readlines(fd)
    for i in 1:length(lines)
        line = lines[i]
        tokens = split(line, ' ')
        if tokens[1] == "delta"
            delta = parse(Float64, tokens[2])
            continue
        end
        step = parse(Int64, tokens[2])
        if tokens[1] == "x"
            # x, y, θ
            hat_xs[step] = [parse(Float64, tokens[3]), parse(Float64, tokens[4]), parse(Float64, tokens[5])] 
        elseif tokens[1] == "z"
            if !haskey(zlist, step)
                zlist[step] = Vector{Vector{Float64}}(undef, 0)
            end
            # ID of landmark, d, ϕ, m
            z = [parse(Float64, tokens[3]), parse(Float64, tokens[4]), parse(Float64, tokens[5]), parse(Float64, tokens[6])]
            push!(zlist[step], copy(z))
        elseif tokens[1] == "u"
            us[step] = [parse(Float64, tokens[3]), parse(Float64, tokens[4])]
        end
    end
    return hat_xs, zlist
end

In [ ]:
hat_xs, zlist = read_data();
xlim = [-5.0, 5.0]
ylim = [-5.0, 5.0]
p = make_ax(xlim, ylim)
p = draw_all(xlim, ylim, hat_xs, zlist, p)
plot(p, legend=nothing)
savefig(p, "images/ch09_graphslam1.png")